### 浅析网络编程

计算机网络是将计算机连接到一起，让网络中的计算机可以通信。实际上在网络编程中，与其说是机器之间的通信不如说是进程之间的通信  
提到网络编程，我们就不得不提到一个概念叫做套接字socket，socket是网络中的一个抽象概念，我们通常用打开一个套接字表示打开一个网络链接而我们想要打开对应的套接字就需要知道对应的对方的IP地址和端口号(进程的地址)  

Socket是网络编程的一个抽象概念。通常我们用一个Socket表示“打开了一个网络链接”，而打开一个Socket需要知道目标计算机的IP地址和端口号，再指定协议类型即可。

一个Socket依赖4项：服务器地址、服务器端口、客户端地址、客户端端口来唯一确定一个Socket。
TCP是建立可靠连接，并且通信双方都可以以流的形式发送数据。相对TCP，UDP则是面向无连接的协议。
使用UDP协议时，不需要建立连接，只需要知道对方的**IP地址和端口号**，就可以直接发数据包。但是，能不能到达就不知道了。
虽然用UDP传输数据不可靠，但它的优点是和TCP比，速度快，对于不要求可靠到达的数据，就可以使用UDP协议。
我们来看看如何通过UDP协议传输数据。和TCP类似，使用UDP的通信双方也分为客户端和服务器。服务器首先需要绑定端口：

127.0.0.1是一个特殊的IP地址，表示本机地址，如果绑定到这个地址，客户端必须同时在本机运行才能连接，也就是说，外部的计算机无法连接进来。

#### 1.socket模块
socket模块帮助我们打开一个网络的套接字（主机+端口）并实现通讯

构建服务器端：
* socket.bind() - 参数是本主机的ip地址和相应要监听的端口的元组
* socket.listen() - 参数是同事的最大挂起的连接的数目，5即可
* socket.accept() - 表示建立连接，返回一个元组，元组的第一个参数是客户端的套接字，第二个参数是对应的上述的的地址元组
* conn.send() - 返回接收的客户端的套接字向客户传送数据，要二进制转码
* conn.close() - 关闭客户的套接字
* gethostname() - 获取主机的ip地址
* gethostbyname() - 根据域名获取ip地址

构建客户端：
* socket(AF_INET,SOCK_STREAM) - 构建一个标准套接字，普通标准套接字可以省略参数
* socket.send() - 返送二进制字符信息
* socket.recv() - 参数是接受的信息的字节数，返回一个二进制字符串，表示接收的信息
* gethostname()
* gethostbyname()
* connect() - 参数是一个元组，第一个是ip地址，第二个是端口号

UDP : socket.recvfrom(size) - return addr / socket.sendto(b'data'  , addr)

In [5]:
# TCP - Client
from socket import *
s = socket()
s.connect(('www.sina.com.cn' , 80))

s.send(b'GET / HTTP/1.1\r\nHost: www.sina.com.cn\r\nConnection: close\r\n\r\n')

buffer = []
while True:
    data = s.recv(1024)
    if not data : break
    else : buffer.append(data)
data = b''.join(buffer)
s.close()

with open("/home/lantian/sina.html" , 'wb') as f:
    header , data = data.split(b'\r\n' , 1)
    f.write(data)

In [ ]:
# TCP - Server
import socket
import threading
import time
def connectclient(s , addr):
    print("Get a new connection from %s:%s " % addr)
    sock.send(b"Hello , I am the server.")
    while True:
        data = sock.recv(1024)
        time.sleep(1)
        if not data or data.decode('utf8') == 'exit' : break
        print("%s" % data.decode('utf8'))
        sock.send(b">>> 1")
    sock.close()
    print("connection to %s:%s is closed!" % addr)
    
if __name__ == "__main__":
    s = socket.socket(socket.AF_INET , socket.SOCK_STREAM)
    s.bind(('127.0.0.1' , 1234))
    s.listen(5)
    print("Waiting for the client...")
    while True:
        sock, addr = s.accept()
        t = threading.Thread(target = connectclient , name = 'subprocessing' , args = (s , addr))
        t.start()

In [ ]:
# TCP - Client
import socket

s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
s.connect(('127.0.0.1' , 1234))

print("-->" + s.recv(1024).decode('utf8'))
s.send(b"I am Client!")
print("-->" + s.recv(1024).decode('utf8'))
s.send(b'exit')
s.close()